In [10]:
from typing import Sequence, Union

import pynmea2
from astropy.time import Time
from pathlib import Path

In [11]:
base_path = Path(".")

msg_types = ["sv_prn_num_", "elevation_deg_", "azimuth_", "snr_"]


In [12]:
def export_snr_data(data: Sequence[Sequence[Union[int, float]]], file_name: str) -> None:
    fmt = (
        "{:3d} "    # SV PRN Number
        "{:8.4f} "  # Elevation in degrees
        "{:10.4f} " # Azimuth in degrees
        "{:10.0f} "   # GPS Time
        "{:6.2f}"   # SNR in dB
    )

    with open(file_name, "w") as f:
        for row in data:
            f.write(fmt.format(*row) + "\n")

In [13]:
log_files = base_path.rglob("*.LOG")
for file_name in log_files:
    file = open(file_name, "r")
    snr_data = []
    current_gps_time = 0
    for line in file.readlines():
        try:
            msg = pynmea2.parse(line)
            if msg.sentence_type == "RMC":
                current_gps_time = Time(msg.datetime).gps
            if msg.sentence_type == "GSV":
                for j in range(1,4):
                    single_data = []
                    do_add = True
                    for k in range(len(msg_types)):
                        if hasattr(msg, f"{msg_types[k]}{j}"):
                            data = getattr(msg, f"{msg_types[k]}{j}")
                            if not data:
                                do_add = False
                                break
                            single_data.append(int(data))
                        else:
                            do_add = False
                            break
                    if not do_add:
                        continue
                    single_data.insert(3, current_gps_time)
                    snr_data.append(single_data)
        except pynmea2.ParseError:
            continue

    export_snr_data(snr_data, "../gnssir/data/"+file_name.stem + ".snr")

In [21]:
msg.timestamp

datetime.time(0, 0, 3, tzinfo=datetime.timezone.utc)